In [ ]:
import subprocess

# Week 9 Data Moshing 

This weeks examples are built from 

https://github.com/tiberiuiancu/datamoshing

and 

https://ffglitch.org/

## Preprocess

It can help to normalise the resolution and other factors on our images so the process works best

In [386]:
def preprocess(v, g="max", scale = [640, 480], fps = 25, trim = "00:00:30"):
    output = v.split(".")[0]+"_processed.mpg" 
    result = subprocess.run(['bin/ffgac',
        '-i', f'{v}',
        '-vf', f'scale={scale[0]}:{scale[1]}',
        '-r', f'{fps}',
        '-t',f'{trim}',
        '-mpv_flags', '+nopimb+forcemv',
        '-qscale:v', "0", 
        '-sc_threshold','max',
        '-g',f'{g}',
        '-vcodec', 'mpeg2video',
        '-y',f'{output}'],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        check=True )
    return result

# # Copy audio stream back from original
def put_back_audio(video, audio):
    output = video.split(".")[0]+"_audio.mpg" 
    result = subprocess.run(['ffmpeg',
                    '-i',f'{video}',
                    '-i',f'{audio}',
                    '-vcodec','mpeg2video',
                    '-c:a','copy',
                    '-map','0:v:0',
                    '-map','1:a:0',
                    "-y",f'{output}'],
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.DEVNULL,
                    check=True )
    result = subprocess.run(['rm',f'{video}'],
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.DEVNULL,
                    check=True )
    return result

In [ ]:
result = preprocess("images/taylor_original.mp4")
result = preprocess("images/taylor_rec.mp4")
result = preprocess("images/advert_bbc4_library_1024x576.mp4")

## Manual Algorithms 

There are two ways we can edit the `motion vectors` of our video

1. By manually editting them. We do this by writing a script which iterates through them and makes any changes to the vertical or horizontal motion of each macroblock in each frame

2. By taking the motion vectors from another video

### Shift  

We provide a separate `.py` file that describes the changes we want to make, in this case `shift.py`. It has **one function** called `mosh_frames(frames)`. Here, we get the motion vectors, have a change to edit them in place, and then return the updated versions.

Here we get the average vertical movement for each frame (across all macroblocks) and then uniform up or down motion based on that

```python
def mosh_frames(frames):
    #step through time
    for frame in frames:
        if not frame:
            continue
        #Get average vertical movement for this frame
        mean_vertical = np.mean(np.array(frame)[:,:,1])
        #step through macroblocks
        for row in frame:
            for mv in row:
                #if average is positive make 2, else make -2
                mv[1] = 2 if mean_vertical > 0 else -2
    return frames
```

### Resetting I-Frames

The more I-Frames we have, the more our glitches get a change to reset 

Normally we would go 

```
I-frame → P-frame₁ → P-frame₂ → P-frame₃
   ↓         ↓         ↓         ↓
Perfect → Accurate → Accurate → Accurate
```

But when we mess with the motion vectors our errors compound
```
I-frame → P-frame₁ → P-frame₂ → P-frame₃
   ↓         ↓         ↓         ↓
Perfect → WRONG → MORE WRONG → VERY WRONG
```

It gets progressively worse until we hit a new I-frame.

1. P-frame₁ uses corrupted motion vectors → pixels end up in wrong places

2. P-frame₂ predicts from the already-corrupted P-frame₁ → compounds the error

3. P-frame₃ predicts from the doubly-corrupted P-frame₂ → even worse

4. Continues until next I-frame which "resets" the corruption

In some contexts, we might want to completely remove any resets to continue our glitching, or we might want to reset occassionaly to get back to the original content. 

We provide the `g` argument to our scripts to control how often we insert I-frames, with `"max"` just being basically none. 

In [ ]:
#File to datamosh
original = "images/taylor_original_processed.mpg"
output = original.split(".")[0]+"_shift.mpg" 
#Path to the script that describes the changes
script = 'shift.py'
#gop_period (the gap between "resetting" I-frames)
g = "100"
result = subprocess.run(['python3',
                'vector_motion.py', f'{original}',
                '-s', f'{script}',
                '-g',f'{g}',
                '-o',output ],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
                check=True)
result = put_back_audio(output,original)

## Removing vertical motion

Here it looks like the mouths aren't moving, because this is vertical motion!

In [410]:
#File to datamosh
original = "images/taylor_original_processed.mpg"
output = original.split(".")[0]+"_horizontal.mpg" 
#Path to the script that describes the changes
script = 'horizontal_motion_example.py'
#gop_period (the gap between "resetting" I-frames)
g = "100"
result = subprocess.run(['python3',
                'vector_motion.py', f'{original}',
                '-s', f'{script}',
                '-g',f'{g}',
                '-o',output ],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
                check=True)
result = put_back_audio(output,original)

## Averaging Motion

In [403]:
result = preprocess("images/advert_bbc4_library_1024x576.mp4")

In [408]:
#File to datamosh
original = "images/advert_bbc4_library_1024x576_processed.mpg"
output = original.split(".")[0]+"_mean.mpg" 
#Path to the script that describes the changes
script = 'average_motion_example.py'
#gop_period (the gap between "resetting" I-frames)
g = "max"
result = subprocess.run(['python3',
                'vector_motion.py', f'{original}',
                '-s', f'{script}',
                '-g',f'{g}',
                '-o',output ],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
                check=True)


## Transferring motion across videos 

In this use case I have made `fft visualisation` of the taylor swift audio in `Dorothy`, then used this as the motion to glitch the original video. 



In [ ]:
#Strip audio
original = "images/taylor_original_processed.mpg"
output = original.split(".")[0]+".wav" 
result = subprocess.run(['ffmpeg',
                '-i',f'{original}',
                f'{output}'],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
                check=True)

In [ ]:
#Transfer motion
#Recording of output from "week9-taylor.py"
motion = "images/taylor_rec_processed.mpg"
original = "images/taylor_original_processed.mpg"
output = original.split(".")[0]+"_transfer.mpg" 
#gop_period (the gap between "resetting" I-frames)
g = "100"
result = subprocess.run(['python3',
                'style_transfer.py',
                #extract motion
                '-e', f'{motion}',
                #transfer to
                '-t', f'{original}',
                '-g',f'{g}',
                f'{output}'],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
                check=True)
result = put_back_audio(output,original)

Or we can use the Taylor Swift motion to move this BBC advert

In [411]:
#Transfer motion
original = "images/advert_bbc4_library_1024x576_processed.mpg"
motion = "images/taylor_original_processed.mpg"
output = original.split(".")[0]+"_transfer.mpg" 
#gop_period (the gap between "resetting" I-frames)
g = "30"
result = subprocess.run(['python3',
                'style_transfer.py',
                #extract motion
                '-e', f'{motion}',
                #transfer to
                '-t', f'{original}',
                '-g',f'{g}',
                f'{output}'],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
                check=True)
result = put_back_audio(output,motion)

# Exploration 

Experiment with different videos and different `gop periods` 

1. Make your own scripts  

    * Random values 

    * Constant values 

    * Changes in scale (amplify movement e.g `mv[0] = 2 * mv[0]`)

    * Different changes based on frame number 

    * Different changes based on location in frame (coordinate)

2. Merge a `Dorothy` sketch with a video 

3. Merge two videos 

4. Make your own videos to merge

## How do I get videos?

